In [21]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

a function to send book tags to the book_details function

In [16]:
def book_tags():
    b_soup = get_soup(url)
    book_tags=b_soup.find_all('article',class_='product_pod')
    books=[]

    for i in book_tags:
        books.append(book_details(i))
    return books

Getting book details such as =>

Book name

Rating

Price of the Book

Link for the book

Generalising these to multiple books so making it as a function

In [12]:
def book_details(book_tag):
    book_title = book_tag.find('a', title=True)['title']
    book_rating = book_tag.find('p')['class'][1]
    book_price = book_tag.find('p', class_='price_color').text[1:]
    book_link = 'http://books.toscrape.com/' + book_tag.find('a')['href']
    return book_title, book_rating, book_price, book_link

This bloack of code gives only the first twenty entries of the home page


In [15]:
url="http://books.toscrape.com/"
response = requests.get(url)

#Converting to BeautifulSoup

b_soup= bs(response.text)
book_list = book_tags();
print(len(book_list))

20


In [18]:
def get_soup(url):
    """Takes URL and returns a soup object"""
    resp = requests.get(url)
    if resp.status_code == 200:
        return bs(resp.text)
    else: return None

def data_collection(page):
    books=[]
    for i in range(1,page+1):
        url= f'http://books.toscrape.com/catalogue/page-{i}.html'
        b_soup=get_soup(url)
        if b_soup:
            try:
                book_tags = b_soup.find_all('article', class_='product_pod')

                for book_tag in book_tags:
                    books.append(book_details(book_tag))
            except:
                print(f'Error reading page {i} . . .')
    books = pd.DataFrame(books,columns=['title','rating','price','link'])
    return books

In [24]:
books_frame = data_collection(50)
books_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1000 non-null   object
 1   rating  1000 non-null   object
 2   price   1000 non-null   object
 3   link    1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


Converting to csv file

In [25]:
books_frame.to_csv('books.csv',index=False)